In [38]:
import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [6]:
from pkg_resources import get_distribution
import platform
print("python", platform.python_version())
print("")
libs = ["numpy", "torch", "torchvision"]
for lib in libs:
    version = get_distribution(lib).version
    print(lib, version)

python 3.5.2

numpy 1.13.1
torch 0.2.0.post1
torchvision 0.1.9


In [ ]:
# ニューラルネットワークで排他的論理和回路のモデル

In [45]:
# モデルクラス定義

class NN(torch.nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        # クラスの初期化
        # :param in_size: 入力層のサイズ
        # :param hidden_size: 隠れ層のサイズ
        # :param out_size: 出力層のサイズ
        super(NN, self).__init__()
        self.xh = torch.nn.Linear(in_size, hidden_size)
        self.hh = torch.nn.Linear(hidden_size, hidden_size)
        self.hy = torch.nn.Linear(hidden_size, out_size)
    
    def __call__(self, x):
        # 順伝播を計算する関数
        # :param x: 入力値
        h = F.relu(self.xh(x))
        h = F.relu(self.hh(h))
        y = F.log_softmax(self.hy(h))
        return y

In [46]:
# 学習

EPOCH_NUM = 2000
HIDDEN_SIZE = 5
BATCH_SIZE = 4
 
# 教師データ
train_data = [
    [[0, 0], [0]],
    [[1, 0], [1]],
    [[0, 1], [1]],
    [[1, 1], [0]]
]
 
# 教師データを変換
in_size = len(train_data[0][0]) #  入力サイズ
out_size = in_size # 出力サイズ
N = len(train_data) # 教師データの総数
train_x, train_t = [], []
for x, t in train_data:
    train_x.append(x)
    train_t.append(t[0])
train_x = np.array(train_x, dtype="float32")
train_t = np.array(train_t, dtype="int32")

# DataLoader化
train = torch.utils.data.TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_t))
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
 
# モデルの定義
model = NN(in_size=in_size, hidden_size=HIDDEN_SIZE, out_size=out_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
 
# 学習開始
print("Train")
st = datetime.datetime.now()
for epoch in range(EPOCH_NUM):
    # ミニバッチ学習
    total_loss = 0
    for i, data in enumerate(train_loader):
        x, t = data
        x, t = Variable(x), Variable(t)
        optimizer.zero_grad()
        y = model(x)
        loss = criterion(y, t)
        total_loss += loss.data[0]
        loss.backward()
        optimizer.step()
    if (epoch+1) % 500 == 0:
        ed = datetime.datetime.now()
        print("epoch:\t{}\ttotal loss:\t{}\ttime:\t{}".format(epoch+1, total_loss, ed-st))
        st = datetime.datetime.now()

Train
epoch:	500	total loss:	0.27735090255737305	time:	0:00:00.495323
epoch:	1000	total loss:	0.05447429418563843	time:	0:00:00.480774
epoch:	1500	total loss:	0.02309884876012802	time:	0:00:00.535039
epoch:	2000	total loss:	0.012654023244976997	time:	0:00:00.525635


In [88]:
# 予測

print("\nPredict")
def predict(model, x):
    x_ = np.array([x], dtype="float32")
    x_ = torch.from_numpy(x_)
    x_ = Variable(x_)
    y = model(x_)
    _, y = torch.max(y.data, 1)
    print("x:\t{}\t => \ty:\t{}".format(x, y[0]))
    
predict(model, [1, 0])
predict(model, [0, 0])
predict(model, [1, 1])
predict(model, [0, 1])


Predict
x:	[1, 0]	 => 	y:	1
x:	[0, 0]	 => 	y:	0
x:	[1, 1]	 => 	y:	0
x:	[0, 1]	 => 	y:	1
